In [18]:
import re
import json
import time
from tqdm.notebook import tqdm
import urllib

In [2]:
with open('../out/camera.html',mode='r', encoding='utf8') as f:
    dat = f.read()

In [3]:
# dat.encode().decode("unicode-escape") # textはutf16にdecodeすると日本語になる

In [4]:
def get_search_value(ptn, str):
    result = re.search(ptn, str)      
    if result:
        return result.group(1)
    else:
        return None

json_str = get_search_value("window._sharedData = (.*);<\/script>", dat)
dat_d = json.loads(json_str)

In [5]:
dat_d.keys()

dict_keys(['config', 'country_code', 'language_code', 'locale', 'entry_data', 'hostname', 'is_whitelisted_crawl_bot', 'connection_quality_rating', 'deployment_stage', 'platform', 'nonce', 'mid_pct', 'zero_data', 'cache_schema_version', 'server_checks', 'knobx', 'to_cache', 'device_id', 'browser_push_pub_key', 'encryption', 'is_dev', 'signal_collection_config', 'consent_dialog_config', 'privacy_flow_trigger', 'rollout_hash', 'bundle_variant', 'frontend_env'])

In [6]:
for key in dat_d.keys():
    temp = dat_d[key]
    if type(temp) == type(dict):
        print(key, len(temp))
    else:
        continue

In [8]:
entry = dat_d["entry_data"]['TagPage'][0]['graphql']['hashtag']

for key in entry.keys():
    temp = entry[key]
    if type(temp) == type(dict):
        print(key, len(temp))
    else:
        print(key,":", )

id :
name :
allow_following :
is_following :
is_top_media_only :
profile_pic_url :
edge_hashtag_to_media :
edge_hashtag_to_top_posts :
edge_hashtag_to_content_advisory :
edge_hashtag_to_related_tags :
edge_hashtag_to_null_state :


In [14]:
# サムネイルの画像url のサンプル抽出
no_post = 0
posts = entry["edge_hashtag_to_media"]["edges"][no_post]["node"]["thumbnail_resources"]
image_url = posts[-1]["src"] # 一番後ろのサムネイルの解像度が一番大きい

# それぞれのedge属性が何枚の画像を持っているか
edges = [
    "edge_hashtag_to_media",
    "edge_hashtag_to_top_posts",
    "edge_hashtag_to_content_advisory",
    "edge_hashtag_to_related_tags",
    "edge_hashtag_to_null_state"]
for e in edges:
    print(e, len(entry[e]["edges"]))

edge_hashtag_to_media 72
edge_hashtag_to_top_posts 9
edge_hashtag_to_content_advisory 0
edge_hashtag_to_related_tags 0
edge_hashtag_to_null_state 0


In [22]:
# urlの抽出
urls = []
for i in range(len(entry["edge_hashtag_to_media"]["edges"])):
    url = entry["edge_hashtag_to_media"]["edges"][i]["node"]["thumbnail_resources"][-1]["src"]
    urls.append(url)

top_urls = []
for i in range(len(entry["edge_hashtag_to_top_posts"]["edges"])):
    url = entry["edge_hashtag_to_top_posts"]["edges"][i]["node"]["thumbnail_resources"][-1]["src"]
    top_urls.append(url)

In [20]:
def download_file(url, dst_path):
    '''
    url : ダウンロードしたい画像/データのurl
    dst_path ： ダウンロードファイルの名前
    '''

    try:
        with urllib.request.urlopen(url) as web_file, open(dst_path, 'wb') as local_file:
            local_file.write(web_file.read())
    except urllib.error.URLError as e:
        print(e)

In [ ]:
for i, im_url in enumerate(tqdm(urls)):
    download_file(im_url, "../out/first_page/image{}.jpg".format(i))
    time.sleep(1)

In [24]:
for i, im_url in enumerate(tqdm(top_urls)):
    download_file(im_url, "../out/first_page/image{}_top.jpg".format(i))
    time.sleep(1)


  0%|          | 0/9 [00:00<?, ?it/s]

In [40]:
with open("../out/posts_笑顔_1000.json", mode="r", encoding="utf8") as f:
    p = json.load(f)

In [41]:
p[list(p.keys())[0]]["src"]

'https://scontent-nrt1-1.cdninstagram.com/v/t51.2885-15/sh0.08/e35/c0.180.1440.1440a/s640x640/242499149_1290419654748202_5544961585097431680_n.jpg?_nc_ht=scontent-nrt1-1.cdninstagram.com&_nc_cat=110&_nc_ohc=7QSwk1q4VacAX_FhNkT&edm=ABZsPhsBAAAA&ccb=7-4&oh=57b4315c24e3281a789f951ff6a4050f&oe=6152F1D4&_nc_sid=4efc9f'

In [46]:
for i, im_key in enumerate(tqdm(p.keys())):
    download_file(p[im_key]["src"], "../out/smile/image{}.jpg".format(i))
    time.sleep(1)

  0%|          | 0/1014 [00:00<?, ?it/s]

KeyboardInterrupt: 